In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import tensorflow as tf
import datetime


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer, SimpleImputer, IterativeImputer

In [1]:
!pip uninstall openpyxl -y
!pip install openpyxl

Found existing installation: openpyxl 3.1.5
Uninstalling openpyxl-3.1.5:
  Successfully uninstalled openpyxl-3.1.5
  Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)


In [3]:
df1=pd.read_csv("Location_information.csv")
df2=pd.read_csv("Weather_data.csv")

In [4]:

combined_df = pd.merge(df1, df2, on='last_updated_epoch', how='left')


In [7]:
combined_df = combined_df.drop(['country', 'location_name'], axis=1)


In [10]:
combined_df=combined_df.drop(['temperature_fahrenheit', 'feels_like_fahrenheit'], axis=1)
combined_df

KeyError: "['temperature_fahrenheit', 'feels_like_fahrenheit'] not found in axis"

In [13]:
combined_df=combined_df.drop(['wind_mph', 'precip_in','pressure_mb','precip_in','visibility_miles','gust_mph'], axis=1)


In [15]:
combined_df=combined_df.drop(['wind_direction', 'timezone'], axis=1)
combined_df

,last_updated_epoch,region,latitude,longitude,last_updated,temperature_celsius,wind_kph,wind_degree,pressure_in,precip_mm,humidity,cloud,feels_like_celsius,visibility_km,uv_index,gust_kph
0,1693000001,Madhya Pradesh,24.57,77.72,29-08-2023 10:45,27.5,20.5,281,29.77,0.00,67,26,29.7,10.0,7,23.8
1,1693000002,Madhya Pradesh,23.33,77.80,29-08-2023 10:45,27.5,15.5,287,29.78,0.00,70,19,30.0,10.0,7,18.0
2,1693000003,Madhya Pradesh,22.07,78.93,29-08-2023 10:45,26.3,18.4,317,29.78,0.00,70,51,28.2,10.0,7,21.2
3,1693000004,Madhya Pradesh,21.86,77.93,29-08-2023 10:45,25.6,16.9,297,29.80,0.00,76,65,27.6,10.0,6,20.9
4,1693000005,Madhya Pradesh,22.75,77.72,29-08-2023 10:45,27.2,16.2,274,29.79,0.00,74,82,29.9,10.0,6,18.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24065,1693024066,Uttar Pradesh,28.88,77.53,11-10-2023 02:45,26.0,6.8,320,29.83,0.00,70,0,26.7,2.5,1,12.9
24066,1693024067,Mizoram,23.97,92.58,11-10-2023 02:45,22.5,3.6,123,29.85,0.01,99,100,25.1,0.0,1,2.2
24067,1693024068,Tamil Nadu,12.93,79.33,11-10-2023 02:45,24.7,3.6,83,29.81,0.28,90,71,27.3,9.0,1,4.9
24068,1693024069,Tamil Nadu,8.97,77.30,11-10-2023 02:45,22.2,10.1,267,29.85,0.03,88,75,24.6,10.0,1,18.5


In [37]:
data.to_csv('merged.csv')

In [29]:
numeric_data = data.select_dtypes(include=[float, int])

# Calculate correlation with temperature_celsius
correlation_with_temperature = numeric_data.corr()['temperature_celsius']

# Print correlations
print(correlation_with_temperature)

last_updated_epoch    -0.226963
latitude              -0.174008
longitude              0.019624
temperature_celsius    1.000000
wind_kph               0.154717
wind_degree            0.121159
pressure_in           -0.562630
precip_mm             -0.008386
humidity              -0.109270
cloud                 -0.024846
feels_like_celsius     0.974471
visibility_km         -0.008533
uv_index               0.287680
gust_kph               0.147298
day_of_year           -0.157612
hour                   0.137262
Name: temperature_celsius, dtype: float64


In [31]:
data=combined_df
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [69]:
data['datetime'] = pd.to_datetime(data['last_updated_epoch'], unit='s')

# Feature engineering
data['day_of_year'] = data['datetime'].dt.dayofyear
data['hour'] = data['datetime'].dt.hour

# Selecting relevant columns
features = ['temperature_celsius', 'day_of_year', 'hour', 'wind_kph', 'pressure_in', 'humidity', 'feels_like_celsius', 'uv_index']

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

In [46]:
# Creating sequences for LSTM
def create_sequences(data, input_steps=1, output_steps=5):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps + 1):
        X.append(data[i:i+input_steps])  # The input sequence (e.g., 1 day)
        y.append(data[i+input_steps:i+input_steps+output_steps, 0])  # Predict the next 5 days' temperatures
    return np.array(X), np.array(y)


time_steps = 1  # Use 1 day to predict the next 5 days
output_steps = 5  # Predict the next 5 days

X, y = create_sequences(scaled_data, time_steps, output_steps)

# Split data into train and test sets
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# X_train and X_test should already have the shape (samples, 1, features)
# y_train and y_test should have the shape (samples, 5)


In [47]:
model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(time_steps, X_train.shape[2])))  # Input shape: (1, features)
model.add(Dense(output_steps))  # Output shape: (5,)
model.compile(optimizer='adam', loss='mse')

C:\Users\pavit\Documents\ML\tf\tfvenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [48]:
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 8s 5ms/step - loss: 0.0856 - val_loss: 0.0078
Epoch 2/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0063 - val_loss: 0.0078
Epoch 3/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0062 - val_loss: 0.0079
Epoch 4/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0060 - val_loss: 0.0069
Epoch 5/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 0.0060 - val_loss: 0.0075
Epoch 6/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0060 - val_loss: 0.0073
Epoch 7/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0071
Epoch 8/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0058 - val_loss: 0.0073
Epoch 9/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0058 - val_loss: 0.0066
Epoch 10/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0058 - val_loss: 0.0069
Epoch 11/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0059 - val_loss: 0.0069
Epoch 12/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

In [49]:
print(X_train)

[[[0.75229358 0.         0.91304348 ... 0.63333333 0.64924115 0.75      ]]

 [[0.75229358 0.         0.91304348 ... 0.66666667 0.65430017 0.75      ]]

 [[0.72477064 0.         0.91304348 ... 0.66666667 0.62394604 0.75      ]]

 ...

 [[0.66743119 1.         0.13043478 ... 0.82222222 0.5851602  0.        ]]

 [[0.71788991 1.         0.13043478 ... 0.56666667 0.61382799 0.        ]]

 [[0.70642202 1.         0.13043478 ... 0.65555556 0.60539629 0.        ]]]


In [76]:
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(np.concatenate((predictions, X_test[:, -1, 1:]), axis=1))[:, 0]
predictions

151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


array([0.68388689, 0.6690675 , 0.69652408, ..., 0.67622149, 0.6791746 ,
       0.67487508])

In [61]:
rmse = np.sqrt(np.mean(predictions - scaler.inverse_transform(scaled_data[train_size+time_steps:len(predictions)+train_size+time_steps])[:, 0])**2)
print(f'RMSE: {rmse}')

RMSE: 0.3210441103056874


In [74]:
predicted_temperatures = model.predict(X_test)
# Assuming scaler was used to scale the original data with 8 features
# Repeat the min and max scalers to match the predicted shape
from sklearn.preprocessing import MinMaxScaler

# Fit scaler on the temperature data from the training set
temperature_scaler = MinMaxScaler()
temperature_scaler.fit(y_train.reshape(-1, 1))


# Predict temperatures
predicted_temperatures = model.predict(X_test)

# Reshape predictions to match scaler input shape
predicted_temperatures = predicted_temperatures.reshape(-1, 1)

# Inverse transform to get original scale
original_scale_predictions = temperature_scaler.inverse_transform(predicted_temperatures)

print("Predicted Temperatures (Original Scale):", original_scale_predictions)




151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
151/151 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Predicted Temperatures (Original Scale): [[0.69041216]
 [0.6941706 ]
 [0.6855072 ]
 ...
 [0.6748308 ]
 [0.68349445]
 [0.69430155]]


In [43]:
import joblib
joblib.dump(model, 'C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/newmodel.pkl')

['C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/newmodel.pkl']

In [78]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Assuming 'data' is a DataFrame containing the raw data
data['datetime'] = pd.to_datetime(data['last_updated_epoch'], unit='s')
data['day_of_year'] = data['datetime'].dt.dayofyear
data['hour'] = data['datetime'].dt.hour

features = ['temperature_celsius', 'day_of_year', 'hour', 'wind_kph', 'pressure_in', 'humidity', 'feels_like_celsius', 'uv_index']
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[features])

def create_sequences(data, input_steps=1, output_steps=5):
    X, y = [], []
    for i in range(len(data) - input_steps - output_steps + 1):
        X.append(data[i:i+input_steps])  # Input sequence
        y.append(data[i+input_steps:i+input_steps+output_steps, 0])  # Next 5 days' temperatures
    return np.array(X), np.array(y)

time_steps = 1
output_steps = 5
X, y = create_sequences(scaled_data, time_steps, output_steps)

train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

model = Sequential()
model.add(LSTM(50, return_sequences=False, input_shape=(time_steps, X_train.shape[2])))
model.add(Dense(output_steps))
model.compile(optimizer='adam', loss='mse')

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Fit scaler only on the temperature column
temperature_scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaler.fit(scaled_data[:, 0].reshape(-1, 1))

# Predict the next 5 days' temperatures
predictions = model.predict(X_test)

# Reverse scaling
# Create an array with the same number of rows as predictions and 1 column for temperatures
predictions_temp = temperature_scaler.inverse_transform(predictions.reshape(-1, 1))

print("Predicted Temperatures (Original Scale):", predictions_temp)



C:\Users\pavit\Documents\ML\tf\tfvenv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.1036 - val_loss: 0.0078
Epoch 2/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0064 - val_loss: 0.0076
Epoch 3/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0059 - val_loss: 0.0072
Epoch 4/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0074
Epoch 5/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0072
Epoch 6/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0068
Epoch 7/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0066
Epoch 8/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0057 - val_loss: 0.0072
Epoch 9/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0066
Epoch 10/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0061 - val_loss: 0.0067
Epoch 11/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 0.0059 - val_loss: 0.0067
Epoch 12/20
602/602 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step

In [79]:
temperature_scaler = MinMaxScaler(feature_range=(0, 1))
temperature_scaler.fit(data[['temperature_celsius']])

# Predict the next 5 days' temperatures
predictions = model.predict(X_test)

# Reshape predictions to be compatible with inverse_transform
predictions_reshaped = predictions.reshape(-1, output_steps)

# Since predictions include multiple days, you need to reverse transform each day
original_scale_predictions = []
for i in range(predictions_reshaped.shape[0]):
    pred = predictions_reshaped[i]
    original_preds = temperature_scaler.inverse_transform(pred.reshape(-1, 1))
    original_scale_predictions.append(original_preds.flatten())

print("Predicted Temperatures (Original Scale):", original_scale_predictions)

151/151 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Predicted Temperatures (Original Scale): [array([24.740492, 24.750624, 24.478025, 24.684237, 24.700998],
      dtype=float32), array([24.048807, 24.244513, 23.84429 , 24.20192 , 24.162777],
      dtype=float32), array([25.083986, 25.129103, 24.94992 , 25.007338, 24.984852],
      dtype=float32), array([24.899357, 24.906332, 24.870262, 24.853931, 24.772503],
      dtype=float32), array([26.203009, 26.22174 , 25.56763 , 25.880575, 25.966005],
      dtype=float32), array([25.919542, 25.919973, 25.471958, 25.677414, 25.773094],
      dtype=float32), array([23.872505, 24.04484 , 23.683237, 23.989254, 23.950655],
      dtype=float32), array([25.551264, 25.592419, 25.285393, 25.378618, 25.384905],
      dtype=float32), array([23.81974 , 24.075844, 23.588564, 23.955269, 23.940676],
      dtype=float32), array([24.770157, 24.79337 , 24.72796 , 24.749304, 24.677797],
      dtype=float32), array([25.016418, 25.02711 , 24.97066 , 24.940605, 24.870266],
    

In [80]:
joblib.dump(model, 'C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/new1model.pkl')

['C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/new1model.pkl']

In [90]:
joblib.dump(scaler, 'C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/scaler.pkl')

['C:/Users/pavit/Documents/ML/HeatAlertSystem/Heatalertproject/myapp/scaler.pkl']